In [1]:
import pandas as pd
import re

In [2]:
# --- Day 7: Handy Haversacks ---
# You land at the regional airport in time for your next flight. 
# In fact, it looks like you'll even have time to grab some food: all flights are currently delayed due to issues in luggage processing.

# Due to recent aviation regulations, many rules (your puzzle input) are being enforced about bags and their contents; bags must be color-coded and must contain specific quantities of other color-coded bags. Apparently, nobody responsible for these regulations considered how long they would take to enforce!

# For example, consider the following rules:

# light red bags contain 1 bright white bag, 2 muted yellow bags.
# dark orange bags contain 3 bright white bags, 4 muted yellow bags.
# bright white bags contain 1 shiny gold bag.
# muted yellow bags contain 2 shiny gold bags, 9 faded blue bags.
# shiny gold bags contain 1 dark olive bag, 2 vibrant plum bags.
# dark olive bags contain 3 faded blue bags, 4 dotted black bags.
# vibrant plum bags contain 5 faded blue bags, 6 dotted black bags.
# faded blue bags contain no other bags.
# dotted black bags contain no other bags.
# These rules specify the required contents for 9 bag types. In this example, every faded blue bag is empty, every vibrant plum bag contains 11 bags (5 faded blue and 6 dotted black), and so on.

# You have a shiny gold bag. If you wanted to carry it in at least one other bag, how many different bag colors would be valid for the outermost bag? (In other words: how many colors can, eventually, contain at least one shiny gold bag?)

# In the above rules, the following options would be available to you:

# A bright white bag, which can hold your shiny gold bag directly.
# A muted yellow bag, which can hold your shiny gold bag directly, plus some other bags.
# A dark orange bag, which can hold bright white and muted yellow bags, either of which could then hold your shiny gold bag.
# A light red bag, which can hold bright white and muted yellow bags, either of which could then hold your shiny gold bag.
# So, in this example, the number of bag colors that can eventually contain at least one shiny gold bag is 4.

# How many bag colors can eventually contain at least one shiny gold bag? (The list of rules is quite long; make sure you get all of it.)

In [3]:
df = pd.read_excel('bags.xlsx')
df.head()

,bags
0,striped white bags contain 4 drab silver bags.
1,drab silver bags contain no other bags.
2,pale plum bags contain 1 dark black bag.
3,"muted gold bags contain 1 wavy red bag, 3 mirr..."
4,"muted teal bags contain 2 pale beige bags, 5 c..."


In [4]:
value = 'striped white bags contain 4 drab silver bags.'

value = re.findall(r'(.*)bags contain(.*)\.',value)

bags = value[0][0]
contents = value[0][1]

print(bags,contents)

striped white   4 drab silver bags


In [5]:
def get_bag_contents(value):
    
    value = re.findall(r'(.*)bags contain(.*)\.',value)

    return value[0][1]

In [6]:
def get_bags(value):
    
    value = re.findall(r'(.*)bags contain(.*)\.',value)

    return value[0][0].strip()

In [7]:
df['contents'] = df.bags.apply(get_bag_contents)
df['bags'] = df.bags.apply(get_bags)

In [8]:
df.head()

,bags,contents
0,striped white,4 drab silver bags
1,drab silver,no other bags
2,pale plum,1 dark black bag
3,muted gold,"1 wavy red bag, 3 mirrored violet bags, 5 bri..."
4,muted teal,"2 pale beige bags, 5 clear beige bags, 2 dott..."


In [9]:
len(df.bags)

594

In [14]:
total_bags = []
small_bags = ['shiny gold']
big_bags = []

getting_bags = True

while getting_bags:

    # get list of bags that contain at least one bag in small_bags (bigger_bags)
    for bag in small_bags:

        # add column 'contains {bag}' to dataframe 
        df[f'contains {bag}'] = df.contents.str.contains(bag)

        # create a dataframe showing only rows with a value of true for 'contains {bag}'
        df_temp = df[df[f'contains {bag}'] == True]

        # create a list of bags using the bags column of the dataframe
        li = list(df_temp.bags)

        # add list of bags to bigger_bags
        big_bags.extend(li)

    # if big_bags id empty break loop
    if len(big_bags) == 0:

        break

    # add bags list to master_list
    total_bags.extend(big_bags)

    # set bags list to equal bigger_bags list
    small_bags = big_bags

    # set bigger_bags to empty
    big_bags = []

# print number of items in total bags
print(len(set(total_bags)))

103


In [11]:
master_list = []
bags = ['shiny gold bag']
bigger_bags = []

getting_bags = True
while getting_bags:
    
    print("master list", master_list)
    print("bags", bags)
    print("bigger_bags", bigger_bags)
    
    # get list of bags that contain at least one bag in bags (bigger_bags)
    for bag in bags:

        # add column contains {bag} for each bag in bags showing if the bag in the bags column contains that bag 
        df[f'contains {bag}'] = df.contents.str.contains(bag)

        # create a list of bags holding each bag that contains a bag in bags.

        df_temp = df[df[f'contains {bag}'] == True]
        
        print(df_temp)

        li = list(df_temp[f'contains {bag}'])
        
        bigger_bags.extend(li)
        
    # if bigger_bags id empty break loop
    
    if len(bigger_bags) == 0:
        
        break
        
    # add bags list to master_list

    master_list.extend(bags)

    # set bags list to equal bigger_bags list

    bags = bigger_bags

    # set bigger_bags to empty

    bigger_bags = []

    #return length of master list    
    len(master_list)

master list []
bags ['shiny gold bag']
bigger_bags []
              bags                                           contents  \
66      dim maroon          5 shiny gold bags, 4 mirrored maroon bags   
85   drab lavender   2 plaid black bags, 4 dotted gray bags, 1 dim...   
434     plaid gold   5 shiny beige bags, 3 faded fuchsia bags, 5 v...   
466    posh salmon   4 dim plum bags, 1 pale yellow bag, 2 shiny g...   
536    dotted lime             1 shiny gold bag, 3 plaid crimson bags   
593    muted white   3 muted tomato bags, 5 light black bags, 4 pa...   

     contains shiny gold bag  contains dim maroon  contains drab lavender  \
66                      True                False                   False   
85                      True                False                   False   
434                     True                False                   False   
466                     True                False                   False   
536                     True                False

TypeError: first argument must be string or compiled pattern

In [ ]:
# identify bags that contain shiny gold bags in the dataframe

df['contains_sgb'] = df.contents.str.contains('shiny gold bag')

df_sgb = df[df.contains_sgb == True]

df_sgb.head()

In [ ]:
# list of bags containing shiny gold bags
bags_containing_sgb = list(df_sgb.bags)

bags_containing_sgb

In [ ]:
# identify bags that contain bags that contain shiny gold bags in the dataframe
for bag in bags_containing_sgb:

    df[f'contains {bag}'] = df.contents.str.contains(bag)

In [ ]:
for bag in contains_sgb:
    
    bags_containing_bags_containing_sgb_list = []
    
    df_bags_containing_bags_containing_sgb = df[df[f'contains {bag}'] == True]
    
    li = list(df_bags_containing_bags_containing_sgb.bags)
    
    print(li)


In [ ]:
for bag in contains_sgb_list:

    df[f'contains {bag}'] = df.contents.str.contains(bag)

In [ ]:
df

In [ ]:
contains_bags_that_contain_sgb = []

for bag in contains_sgb_list:
    
    df[f'contains {bag}'] = df.contents.str.contains(bag)
    
    df_temp = df[df[f'contains {bag}'] == True]
    
    li = list(df.bags)
    
    contains_bags_that_contain_sgb.extend(li)

contains_bags_that_contain_sgb = set(contains_bags_that_contain_sgb)

contains_bags_that_contain_sgb

In [ ]:
print(len(contains_bags_that_contain_sgb), len(set(contains_bags_that_contain_sgb)))

In [ ]:
df.shape

In [ ]:
contains_bags_that_contain_bags_that_contain_sgb = []

for bag in contains_bags_that_contain_sgb:
    
    df_temp = df[df[f'contains {bag}'] == True]
    
    li = list(df.bags)
    
    contains_bags_that_contain_bags_that_contain_sgb.extend(li)

contains_bags_that_contain_bags_that_contain_sgb = set(contains_bags_that_contain_bags_that_contain_sgb)

contains_bags_that_contain_bags_that_contain_sgb


